<a href="https://www.kaggle.com/code/joshuaokolo/terrorism-finance-prevention?scriptVersionId=104054699" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Load Global Terrorism Dataset

In [ ]:
import pandas as pd
from neo4j import GraphDatabase
from loguru import logger
import os
from pathlib import Path

project_dir = str(Path(__file__).resolve().parents[0])
data_dir = os.path.join(project_dir, "data")
logger.add(data_dir+'Perp_Event_Relation.log')
logger.info('Dataset loading')
GTD_df = pd.read_csv(data_dir+("../input/gtd/globalterrorismdb_0718dist.csv"))
GTD_df_small = GTD_df[GTD_df["iyear"] >= 2010]
logger.info('The dataset is loaded')

uri="bolt://server_address:7687"
user, password=("username", "password")
database_name="databaseName"

driver = GraphDatabase.driver(uri, auth=(user, password))
logger.info('Connected to database')
query_create_rel_att="""
MATCH (e:Event {eventid:$eid}),(p:Perpetrator {name:$name_perpetrator})
MERGE (e)<-[r:IS_RESPONSIBLE_FOR] - (p)
ON CREATE SET r.Claimed=$claimed
"""

loaded = 0
nn=0
with driver.session(database=database_name) as session:       
    for ix,row in GTD_df_small.iterrows():
        nn+=1
        if(nn%1000 ==0):
            logger.info(str(nn)+'Loaded')
            session.run(query_create_rel_att, eid=row["eventid"], 
                  claimed=row["claimed"], name_perpetrator=row["gname"])   
        non_nan_row_entries = row.isna()
        if not non_nan_row_entries["gname2"] :
            session.run(query_create_rel_att, eid=row["eventid"],
                  claimed=row["claim2"], name_perpetrator=row["gname2"])
        non_nan_row_entries = row.isna()
        if not non_nan_row_entries["gname3"] :
            session.run(query_create_rel_att, eid=row["eventid"],
                  claimed=row["claim3"], name_perpetrator=row["gname3"])

logger.info('Total loaded'+ str(nn))

## Sanction Lists

In [ ]:
LOAD CSV WITH HEADERS FROM 'file:///Sanction_List.csv' AS row WITH row.NAME AS name, 
row.NATIONALITY AS nationality, row.INDIVIDUAL_ALIAS AS Alias, 
row.INDIVIDUAL_ADDRESS AS address, row.INDIVIDUAL_DATE_OF_BIRTH AS DateOfBirth, 
row.COMMENTS AS comments

WHERE row.TYPE ='Individual' 

MERGE (i:individual {name: name})

SET i.nationality = nationality,i.Alias = Alias,i.address = address,
i.DateOfBirth = DateOfBirth ,i.comments = comments

RETURN count(i)

In [ ]:
LOAD CSV WITH HEADERS FROM 'file:///Sanction_List.csv' AS row WITH row.NAME AS name, row.LIST_TYPE AS listType

MATCH (i:individual {name: name}) 

MATCH (n:sancAuth {type: listType})

MERGE (i)-[rel:SanctionBy]->(n)

RETURN count(rel)

## Panama Papers

Code is for querying Neo4j

In [ ]:
:auto USING PERIODIC COMMIT 10000  
LOAD CSV WITH HEADERS FROM "file:///panama_papers.nodes.addressm.csv" AS row  
MERGE (n:Node {node_id:row.node_id})  
ON CREATE SET n = row, n:Address;

## Twitter Data

Code is for querying Neo4j

In [ ]:
LOAD CSV WITH HEADERS FROM "file:///TwitterDataForaEvent.csv" 
AS row WITH row WHERE toFloat(row.sentiment_analysis) > 0.4 
MERGE (n:Tweet {name : row.name}) 
SET n.tweet=row.tweet,n.username = row.username,n.sentiment_score=row.sentiment_analysi,
n.tweet_date=row.date,n.link=row.link,n.attack_ref=row.attack 
RETURN count(*)

In [ ]:
MATCH (t:Tweet {attack_ref:'2019-01-09-West Bank-Jerusalem'}) 
MATCH (n:Event)-[:EVENT_LOCATED_IN]->(r)-[]->(c) 
WHERE n.iyear=2019 AND n.imonth=1 AND n.iday=9 AND c.provstate='West Bank' 
MERGE (n)<-[rel:Tweet_is_about]-(t) 
RETURN count(*)

In [ ]:
MATCH (n:individual),(o:Officer) WHERE toLower (n.name) = toLower (o.name) 
CREATE (n)-[r:sancIndividual]->(o) 
RETURN r

MATCH (e:Entity) MATCH (o:org) WHERE toLower(e.name) = toLower(o.name) 
CREATE (e)-[r:sancOrg]->(o) 
RETURN r

In [ ]:
MATCH p=()-[r:IS_COLLABORATOR]->() RETURN p LIMIT 25

In [ ]:
MATCH p=()-[r:HAS_SIMILAR_NAME]->() RETURN p LIMIT 25

In [ ]:
MATCH (n:Tweet)-[]->(e:Event)-->(p:Perpetrator) where p.name <> 'Unknown' 
RETURN n,e,p

In [ ]:
MATCH (a:Officer),(b:individual) WHERE a.name CONTAINS 'NIGEL RICHARD JAMES COWIE' 
MATCH p=allShortestPaths((a)-[:officer_of|intermediary_of|registered_address|sancIndividual*..10]-(b)) 
RETURN p

In [ ]:
Match (p:Perpetrator), (e:entity) WITH p,e 
MATCH r=allShortestPaths( (p) -[rel:IS_OWNED_BY|GSTransaction*..9]- (e)) 
return r

## Streamlit Dashboard

In [ ]:
import streamlit as st
from neo4j import GraphDatabase 
import pandas as pd 
from streamlit_agraph import agraph, TripleStore, Config

In [ ]:
store = TripleStore() 
store.add_triple(node1, link, node2)

In [ ]:
result2 = session.run("""MATCH (n:Officer)-[rel1]->(r:Entity)<-[rel2]- 
(n1:Officer)<-[rel3:sancIndividual]-(n2)-[rel4:SanctionBy]->(n3) 
WHERE n.name = '""" + name + """' 
RETURN n,r,n1,n2,n3,rel1,rel2,rel3,rel4""") 
rList = result2.data()

In [ ]:
config = Config(height=500, width=700, nodeHighlightBehavior=True, 
highlightColor="#F7A7A6", directed=True, collapsible=True, 
node={'labelProperty': 'label'},link={'labelProperty': 'label', 'renderLabel': True}) 
agraph(list(store.getNodes()), list(store.getEdges()), config)

## References

https://riptutorial.com/neo4j/example/13244/installation---starting-a-neo4j-server

https://www.start.umd.edu/gtd

https://connected-data.london/sessions/using-graphs-to-improve-machine-learning-and-produce-explainable-ai/

https://omdena.com/blog/knowledge-graph-to-unveil-terrorist-organizations-and-financing-partners/

https://home.treasury.gov/policy-issues/financial-sanctions/consolidated-sanctions-list-non-sdn-lists

https://www.international.gc.ca/world-monde/international_relations-relations_internationales/sanctions/consolidated-consolide.aspx?lang=eng

https://www.gov.uk/government/publications/the-uk-sanctions-list

https://offshoreleaks.icij.org/

https://neo4j.com/resources/